# EPH - Individuos


## Ajuste de ruta del proyecto en Notebook

In [1]:
import sys
import csv
from pathlib import Path

# Ajusto para que la reconozca la ruta del proyecto
sys.path.append("..") 

#CONSTANTS
from src.utils.constants import PROJECT_ROOT, DATA_SOURCE_DIR,DATA_CLEAN_DIR,FILENAME_INDIVIDUOS_UNIFIED
print(PROJECT_ROOT)

C:\Users\PC\Documents\1) Estudios y Programacion\EncuestAR


## Unificación de los archivos de Individuos

In [2]:
from src.utils.helpers import  process_file, save_to_txt


# Proceso todos los archivos de individuos
individual=process_file(DATA_SOURCE_DIR, "individual")

# Guardo contenido en un archivo txt en la ruta DATA_CLEAN_DIR con el nombre de la constante FILENAME_INDIVIDUOS_UNIFIED
save_to_txt(individual, DATA_CLEAN_DIR, FILENAME_INDIVIDUOS_UNIFIED)

C:\Users\PC\Documents\1) Estudios y Programacion\EncuestAR\data\raw\usu_hogar_T324.txt
C:\Users\PC\Documents\1) Estudios y Programacion\EncuestAR\data\raw\usu_individual_T324.txt
✅ Archivo TXT guardado en: C:\Users\PC\Documents\1) Estudios y Programacion\EncuestAR\data\clean\individuos_unificado.txt


## Procesamiento de los archivos

Abro el archivo unificado

In [3]:
from src.utils.constants import INDIVIDUOS_UNIFIED_DIR
from src.utils.helpers import read_file_dic
import csv

header,data=read_file_dic(INDIVIDUOS_UNIFIED_DIR)

Proceso el archivo

In [4]:
import src.procesamientos.individuos as indiv

# Agrego las nuevas columnas al header
header.extend(["CH04_str","NIVEL_ED_str","CONDICION_LABORAL","UNIVERSITARIO"])

# Proceso las nuevas columnas por cada fila
for row in data:
    indiv.add_data_ch04str(row)
    indiv.add_data_nivel_ed_str(row)
    indiv.add_data_cond_lab(row)
    indiv.add_data_universitario(row)

Guardo los datos procesados a un archivo

In [5]:
from src.utils.constants import INDIVIDUOS_PROCESSED_DIR

with open (INDIVIDUOS_PROCESSED_DIR,"w",encoding="UTF-8",newline='') as file_individuos_processed:
    csv_writer= csv.DictWriter(file_individuos_processed, delimiter=";",fieldnames=header)
    csv_writer.writeheader()
    csv_writer.writerows(data)

## Análisis de Individuos

Abro el archivo de individuos procesado:

In [6]:
header,data=read_file_dic(INDIVIDUOS_PROCESSED_DIR)

### 1- A partir de la información de cada año contenida en el dataset se debe informar, año tras año, el porcentaje de personas mayores a 2 años capaces e incapaces de leer y escribir.

In [7]:
indiv.cant_personas_alfabetizadas(data)

Año	Alfabetizados	No alfabetizados
2024	27519850		1647765


### 2- A partir de un año y trimestre elegido por el usuario informar el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior

In [8]:
#Ingreso de datos por el usuario
ano=input("Ingrese el año: ")
trim=input("Ingrese el trimestre [1 a 4]: ")

indiv.porc_extranjero_universitario(ano,trim,data)

ZeroDivisionError: division by zero

### 3- A partir de la información contenida en el dataset informar el año y trimestre donde hubo menor desocupación.

In [ ]:
indiv.info_menor_desocupacion(data)

Valor mínimo de desocupación: 994334 en los siguientes años y trimestres:
Año: 2024, Trimestre: 3
